<br>
# **Phrase 분석**

<br>
## **1 불용어 처리**
Stop Words

In [1]:
texts = 'I like such a Wonderful Snow Ice Cream'
texts = texts.lower()
texts

'i like such a wonderful snow ice cream'

In [2]:
from nltk import word_tokenize
tokens = word_tokenize(texts)
tokens

['i', 'like', 'such', 'a', 'wonderful', 'snow', 'ice', 'cream']

In [3]:
from nltk.corpus import stopwords
stopwords.words('english')[::18]

['i', 'her', 'those', 'an', 'into', 'further', 'such', 'now', 'mightn']

In [4]:
tokens = [word   for word in tokens   
                 if word not in stopwords.words('english')]
print(tokens)

['like', 'wonderful', 'snow', 'ice', 'cream']


<br>
## **2 한글의 불용어 처리**

In [5]:
f = open('./data/한국어불용어100.txt', 'r')
s = f.read(); f.close()

stop_words = [ txt.split('\t')[:2]  for txt in s.split('\n') ]
stopword   = {}
for txt in stop_words:
    try:    stopword[txt[0]] = txt[1]
    except: pass
stopword

{'이': 'NP',
 '있': 'VA',
 '하': 'VV',
 '것': 'NNB',
 '들': 'VV',
 '그': 'MM',
 '되': 'VV',
 '수': 'NNB',
 '보': 'VX',
 '않': 'VX',
 '없': 'VA',
 '나': 'VX',
 '사람': 'NNG',
 '주': 'VV',
 '아니': 'VCN',
 '등': 'NNB',
 '같': 'VA',
 '우리': 'NP',
 '때': 'NNG',
 '년': 'NNB',
 '가': 'VV',
 '한': 'MM',
 '지': 'VX',
 '대하': 'VV',
 '오': 'VV',
 '말': 'VX',
 '일': 'NNB',
 '그렇': 'VA',
 '위하': 'VV',
 '때문': 'NNB',
 '그것': 'NP',
 '두': 'VV',
 '말하': 'VV',
 '알': 'VV',
 '그러나': 'MAJ',
 '받': 'VV',
 '못하': 'VX',
 '그런': 'MM',
 '또': 'MAG',
 '문제': 'NNG',
 '더': 'MAG',
 '사회': 'NNG',
 '많': 'VA',
 '그리고': 'MAJ',
 '좋': 'VA',
 '크': 'VA',
 '따르': 'VV',
 '중': 'NNB',
 '나오': 'VV',
 '가지': 'VV',
 '씨': 'NNB',
 '시키': 'XSV',
 '만들': 'VV',
 '지금': 'NNG',
 '생각하': 'VV',
 '그러': 'VV',
 '속': 'NNG',
 '하나': 'NR',
 '집': 'NNG',
 '살': 'VV',
 '모르': 'VV',
 '적': 'XSN',
 '월': 'NNB',
 '데': 'NNB',
 '자신': 'NNG',
 '안': 'MAG',
 '어떤': 'MM',
 '내': 'VV',
 '경우': 'NNG',
 '명': 'NNB',
 '생각': 'NNG',
 '시간': 'NNG',
 '그녀': 'NP',
 '다시': 'MAG',
 '이런': 'MM',
 '앞': 'NNG',
 '보이': 'VV',
 '번': '

<br>
## **3 Token 객체를 활용한 통계적 분석**
1. 레벤슈타인의 편집거리
1. Accuracy

In [6]:
text1 = "자연 언어에 대한 연구는 오래전부터 이어져 오고 있음에도 2018년까지도 사람처럼 이해하지는 못한다.".split()
text2 = "자연 언어에 대한 연구는 오래전부터 이어져 들어서도 아직 컴퓨터가 사람처럼 이해하지는 못한다.".split()
text3 = "자연 아직 컴퓨터가 언어에 들어서도 못한다 이어져 사람처럼 이해하지는 대한 연구는 오래전부터.".split()
len(text1), len(text2), len(text3)

(12, 12, 12)

In [7]:
from nltk.metrics import edit_distance
edit_distance('파이썬 알고리즘', '파파미 알탕')

5

In [8]:
print('생략된 단어가 다를 때 : {} \n어휘 순서를 바꿨을 때 : {}'.format(
    edit_distance(text1, text2), 
    edit_distance(text2, text3)))

생략된 단어가 다를 때 : 3 
어휘 순서를 바꿨을 때 : 10


In [9]:
# 02 accuracy 정확도 측정
from nltk.metrics import accuracy
accuracy('파이썬', '파이프')

0.6666666666666666

In [10]:
print('생략된 단어가 다를 때 {:.4} \n어휘 순서를 바꿨을 때 {:.4}'.format(
    accuracy(text1, text2), 
    accuracy(text2, text3)))

생략된 단어가 다를 때 0.75 
어휘 순서를 바꿨을 때 0.08333


<br>
## **4 Token 고유 객체를 활용한 통계적 검증방법**
1. precision = Correct / (Correct + Incorrect + Missing)
1. recall = Correct / (Correct + Incorrect + Spurious<가짜>)
1. f_measure = (2 X Precision X Recall) / (Precision + Recall)

In [11]:
text1 = set(text1)
text2 = set(text2)
text3 = set(text3)
len(text1), len(text2), len(text3)

(12, 12, 12)

In [12]:
from nltk.metrics import precision
precision({'파이썬'}, {'파르썬'})

0.0

In [13]:
print('생략된 단어가 다를 때 {:.4} \n어휘 순서를 바꿨을 때 {:.4}'.format(
    precision(set(text1), set(text2)), 
    precision(set(text2), set(text3))))

생략된 단어가 다를 때 0.75 
어휘 순서를 바꿨을 때 0.8333


In [14]:
from nltk.metrics import recall
print('생략된 단어가 다를 때 {:.4} \n어휘 순서를 바꿨을 때 {:.4}'.format(
    recall(text1, text2), 
    recall(text2, text3)))

생략된 단어가 다를 때 0.75 
어휘 순서를 바꿨을 때 0.8333


In [15]:
from nltk.metrics import f_measure
print('생략된 단어가 다를 때 {:.4} \n어휘 순서를 바꿨을 때 {:.4}'.format(
    f_measure(text1, text2), 
    f_measure(text2, text3)))

생략된 단어가 다를 때 0.75 
어휘 순서를 바꿨을 때 0.8333


<br>
# **3 N-gram 의 활용**
<br>
## **1 N-gram 생성하기**

In [16]:
# 독일 퀘르버 재단 연설문 : 베를린 선언
f     = open('./data/베를린선언.txt', 'r')
texts_org = f.read()
f.close()

In [17]:
from nltk.tokenize import word_tokenize
word_token = word_tokenize(texts_org)

In [18]:
from nltk.util import ngrams

texts_sample = [txt for txt in ngrams(word_token, 3)]
texts_sample[:5]

[('존경하는', '독일', '국민'),
 ('독일', '국민', '여러분'),
 ('국민', '여러분', ','),
 ('여러분', ',', '고국에'),
 (',', '고국에', '계신')]

<br>
## **2 Point wise Mutual Information**
PMI

In [19]:
from nltk.tokenize import RegexpTokenizer
re_capt = RegexpTokenizer('[가-힣]\w+')
raw_texts = re_capt.tokenize(texts_org)
raw_texts[:10]

['존경하는', '독일', '국민', '여러분', '고국에', '계신', '국민', '여러분', '하울젠', '쾨르버재단']

In [20]:
texts = ''
for txt in raw_texts:
    texts += txt + " "
texts[:200]

'존경하는 독일 국민 여러분 고국에 계신 국민 여러분 하울젠 쾨르버재단 이사님과 모드로 동독 총리님을 비롯한 내외 귀빈 여러분 먼저 냉전과 분단을 넘어 통일을 이루고 힘으로 유럽통합과 국제평화를 선도하고 있는 독일과 독일 국민에게 무한한 경의를 표합니다 오늘 자리를 마련해 주신 독일 정부와 쾨르버 재단에도 감사드립니다 아울러 얼마 별세하신 헬무트 총리의 가족'

In [21]:
%%time
# 베를린 선언문에 Tag 속성 추가하기
from konlpy.tag import Twitter
twitter = Twitter()
tagged_words = twitter.pos(texts)

CPU times: user 4.6 s, sys: 93 ms, total: 4.69 s
Wall time: 1.64 s


<br>
## **2-1 Bi-Gram 을 대상으로 한 PMI**
최상위 우도값 10개를 추출한다

In [22]:
from nltk import collocations

finder = collocations.BigramCollocationFinder.from_words(tagged_words)
finder

In [23]:
measures = collocations.BigramAssocMeasures()
finder.nbest(measures.pmi, 10)

[(('가능하며', 'Adjective'), ('불가', 'Noun')),
 (('가스', 'Noun'), ('관', 'Noun')),
 (('가운데', 'Noun'), ('현재', 'Noun')),
 (('감사', 'Noun'), ('드립니', 'Verb')),
 (('갖춰', 'Verb'), ('지', 'PreEomi')),
 (('같은', 'Adjective'), ('공감', 'Noun')),
 (('거나', 'Eomi'), ('깨져', 'Verb')),
 (('건너지', 'Verb'), ('않기', 'Verb')),
 (('걷어', 'Verb'), ('차는', 'Verb')),
 (('검증', 'Noun'), ('가능하며', 'Adjective'))]

<br>
## **2-2 Tri-Gram 을 대상으로한 PMI**

In [24]:
finder = collocations.TrigramCollocationFinder.from_words(tagged_words)
finder

In [25]:
measures = collocations.TrigramAssocMeasures()
finder.nbest(measures.pmi, 10)

[(('가능하며', 'Adjective'), ('불가', 'Noun'), ('역적', 'Noun')),
 (('가스', 'Noun'), ('관', 'Noun'), ('연결', 'Noun')),
 (('가운데', 'Noun'), ('현재', 'Noun'), ('생존', 'Noun')),
 (('같은', 'Adjective'), ('공감', 'Noun'), ('대', 'Suffix')),
 (('거나', 'Eomi'), ('깨져', 'Verb'), ('서도', 'Noun')),
 (('검증', 'Noun'), ('가능하며', 'Adjective'), ('불가', 'Noun')),
 (('견', 'Noun'), ('지하', 'Noun'), ('면서', 'Noun')),
 (('과도', 'Josa'), ('같은', 'Adjective'), ('공감', 'Noun')),
 (('들어서는', 'Verb'), ('대전', 'Noun'), ('환', 'Noun')),
 (('록', 'Eomi'), ('앞장서서', 'Verb'), ('돕겠', 'Verb'))]

<br>
# **4 은닉 마르코프 모델 추정**
Hidden Markov Model estimation

<br>
## **1 HMM 알고리즘을 활용**
문장분석

In [26]:
texts = texts_org.split('\n\n')
texts[:10]

['존경하는 독일 국민 여러분,',
 '고국에 계신 국민 여러분,',
 '하울젠 쾨르버재단 이사님과 모드로 전 동독 총리님을 비롯한 내외 귀빈 여러분,',
 '\n먼저, 냉전과 분단을 넘어 통일을 이루고,',
 '그 힘으로 유럽통합과 국제평화를 선도하고 있는',
 '독일과 독일 국민에게 무한한 경의를 표합니다.',
 '\n오늘 이 자리를 마련해 주신',
 '독일 정부와 쾨르버 재단에도 감사드립니다.',
 '\n아울러, 얼마 전 별세하신 故 헬무트 콜 총리의 가족과 ',
 '독일 국민들에게 깊 은 애도와 위로의 마음을 전합니다.']

In [27]:
texts = [text.replace('\n', '')  for text in texts]
texts = [text.replace(',', '')  for text in texts]
texts = [text.replace('.', '')  for text in texts]
texts[:10]

['존경하는 독일 국민 여러분',
 '고국에 계신 국민 여러분',
 '하울젠 쾨르버재단 이사님과 모드로 전 동독 총리님을 비롯한 내외 귀빈 여러분',
 '먼저 냉전과 분단을 넘어 통일을 이루고',
 '그 힘으로 유럽통합과 국제평화를 선도하고 있는',
 '독일과 독일 국민에게 무한한 경의를 표합니다',
 '오늘 이 자리를 마련해 주신',
 '독일 정부와 쾨르버 재단에도 감사드립니다',
 '아울러 얼마 전 별세하신 故 헬무트 콜 총리의 가족과 ',
 '독일 국민들에게 깊 은 애도와 위로의 마음을 전합니다']

In [28]:
tagged_words = [twitter.pos(text)  for text in texts]
tagged_words[:2]

[[('존경하는', 'Verb'), ('독일', 'Noun'), ('국민', 'Noun'), ('여러분', 'Noun')],
 [('고국', 'Noun'),
  ('에', 'Josa'),
  ('계신', 'Verb'),
  ('국민', 'Noun'),
  ('여러분', 'Noun')]]

In [29]:
from nltk.util import unique_list
tag_set = unique_list( tag     for sent        in  tagged_words     
                               for (word, tag) in  sent )
print(len(tag_set))
tag_set

16


['Verb',
 'Noun',
 'Josa',
 'Suffix',
 'Adjective',
 'Eomi',
 'Exclamation',
 'Foreign',
 'Determiner',
 'Number',
 'Punctuation',
 'Alpha',
 'PreEomi',
 'Adverb',
 'Conjunction',
 'VerbPrefix']

In [30]:
symbols = unique_list( word     for sent        in  tagged_words    
                                for (word, tag) in  sent )
print(len(symbols))
symbols[:10]

987


['존경하는', '독일', '국민', '여러분', '고국', '에', '계신', '하울', '젠', '쾨르버']

In [31]:
%%time
# 히든 마르코프 훈련모듈 활성화 (16개 tag,  987개 문장) 
def train_and_test(tagged_words, est):
    from nltk.util import unique_list
    tag_set = unique_list( tag   for sent        in  tagged_words     
                                 for (word, tag) in  sent )
    symbols = unique_list( word  for sent        in  tagged_words    
                                 for (word, tag) in  sent )
    import nltk
    trainer = nltk.tag.HiddenMarkovModelTrainer(tag_set, symbols)
    train_corpus, test_corpus = [], []  
    for i in range(len(tagged_words)) :
        if i % 10 :  
            train_corpus += [tagged_words[i]]
        else :       
            test_corpus += [tagged_words[i]]
    hmm = trainer.train_supervised(train_corpus, estimator = est)
    print('%.2f%%' % (100 * hmm.evaluate(test_corpus)))

train_and_test(tagged_words, None)

44.69%
CPU times: user 196 ms, sys: 0 ns, total: 196 ms
Wall time: 70.5 ms


<br>
## **2 HMM 영문분석**
Brown 어휘목록을 활용

In [32]:
%%time
def HMM_english():
    import nltk
    cor = nltk.corpus.brown.tagged_sents(categories='adventure')[:500]

    from nltk.util import unique_list
    tag_set = unique_list( tag  for sent        in  cor     
                                for (word, tag) in  sent )
    symbols = unique_list( word  for sent        in  cor    
                                 for (word, tag) in  sent )
    print("tag set  : {} \nWord set : {}".format(len(tag_set), len(symbols)))

    train_corpus, test_corpus = [], []  
    trainer = nltk.tag.HiddenMarkovModelTrainer(tag_set, symbols)
    for i in range(len(cor)) :
        if i % 10 :  train_corpus += [cor[i]]   # train 90% , test 10% 데이터 생성
        else :       test_corpus += [cor[i]]
    hmm = trainer.train_supervised(train_corpus, estimator=None)  # .train_supervised()
    print('%.2f%%' % (100 * hmm.evaluate(test_corpus)))

HMM_english()

tag set  : 92 
Word set : 1464
22.75%
CPU times: user 573 ms, sys: 15.7 ms, total: 589 ms
Wall time: 507 ms


<br>
# **5 TF-IDF**
Term Frequency-Inverse Document Frequency

<br>
## **1 영문 데이터 전처리**
트럼프 취임사 연설문

In [33]:
f = open('./data/trump.txt', 'r')
texts_org = f.read()
f.close()

from nltk import word_tokenize
texts = word_tokenize(texts_org)
texts[:5]

['Chief', 'Justice', 'Roberts', ',', 'President']

In [34]:
from nltk.corpus import stopwords
stopword_eng = stopwords.words('english')

import string
punct = string.punctuation
punct = [punct[i] for i in range(len(punct))]
punct = punct + stopword_eng + ['\n'] 
len(punct)

186

In [35]:
texts = [txt.lower()    for txt in texts   if txt.lower() not in punct]
document = ''
for txt in texts:
    document += txt + ' '

<br>
## **2 tf idf **
연설문내 단어들의 빈도를 재조정

In [36]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vec   = TfidfVectorizer()
transformed = tfidf_vec.fit_transform(raw_documents = [document])
index_value = {i[1]:i[0] for i in tfidf_vec.vocabulary_.items()}

transformed 

CPU times: user 4.26 ms, sys: 11 µs, total: 4.27 ms
Wall time: 4.04 ms


In [37]:
fully_indexed = []

import numpy as np
transformed = np.array(transformed.todense())

for row in transformed:
    fully_indexed.append({index_value[column]:value for (column,value) in enumerate(row)})

In [38]:
import pandas as pd
tfidf = pd.Series(fully_indexed[0]).sort_values(ascending=False)
tfidf[:15]

america     0.423619
american    0.232990
people      0.211809
country     0.190628
nation      0.190628
one         0.169447
every       0.148266
never       0.127086
world       0.127086
back        0.127086
new         0.127086
great       0.127086
make        0.105905
god         0.105905
today       0.105905
dtype: float64

In [39]:
tfidf['great']

0.12708556268223822